In [1]:
import logging
import random
import warnings
from datetime import datetime

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import StratifiedShuffleSplit
from torch import nn
from torchvision import models, transforms

In [2]:
%matplotlib widget

In [3]:
warnings.simplefilter('ignore')

In [4]:
logger = logging.getLogger(__name__)
np.random.seed(2019)

log_path = datetime.now().strftime('./logs/%Y-%m-%d-%H-%M-%S.log')
logging.basicConfig(filename=log_path, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler())

In [5]:
imagenet_img_channels = 3
imagenet_img_dim = 224
mnist_modified_img_dim = 64
pad = (imagenet_img_dim - mnist_modified_img_dim) // 2

num_classes = 10
tr_samples = 40000
ts_samples = 10000

In [6]:
x_tr = np.expand_dims(pd.read_pickle('./dataset/train_images.pkl'), axis=1)
x_ts = np.expand_dims(pd.read_pickle('./dataset/test_images.pkl'), axis=1)
y_tr = pd.read_csv('./dataset/train_labels.csv').values[:, 1]

In [7]:
print(x_tr.shape)
print(x_ts.shape)
print(y_tr.shape)

(40000, 1, 64, 64)
(10000, 1, 64, 64)
(40000,)


In [8]:
plt.figure()
plt.suptitle('Class Distribution Histogram')
plt.xlabel('Class')
plt.ylabel('Count')
plt.xticks(range(num_classes + 1))
plt.hist(y_tr, bins=range(num_classes + 1), histtype='bar', rwidth=0.9)
plt.show()

FigureCanvasNbAgg()

In [9]:
# Threshold
x_tr_org = np.copy(x_tr)
x_tr[x_tr > 200] = 255
x_tr[x_tr < 255] = 0

x_ts[x_ts > 200] = 255
x_ts[x_ts < 255] = 0

In [10]:
num_random_samples = (5, 5)
idx = np.random.randint(0, x_tr.shape[0], num_random_samples)

In [11]:
plt.figure()
plt.suptitle('Random samples before thresholding')
for i in range(num_random_samples[0]):
    for j in range(num_random_samples[1]):
        plt.subplot(num_random_samples[0], num_random_samples[1], i * num_random_samples[1] + j + 1)
        plt.imshow(x_tr_org[idx[i, j]].squeeze().squeeze(), cmap='gray')
plt.show()

plt.figure()
plt.suptitle('Random samples after thresholding')
for i in range(num_random_samples[0]):
    for j in range(num_random_samples[1]):
        plt.subplot(num_random_samples[0], num_random_samples[1], i * num_random_samples[1] + j + 1)
        plt.imshow(x_tr[idx[i, j]].squeeze().squeeze(), cmap='gray')
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [12]:
del x_tr_org

In [13]:
# Remove dots
x_tr_org = np.copy(x_tr)
for i, x_i in enumerate(x_tr):
    x_i = x_i.squeeze().astype(np.uint8)
    _, output, stats, _ = cv2.connectedComponentsWithStats(x_i, connectivity=8)

    min_component_size = 50

    x_i_rm = np.zeros(output.shape)
    for j, size in enumerate(stats[1:, -1]):  # Ignore the background
        if size >= min_component_size:
            x_i_rm[output == j + 1] = 255
    x_tr[i] = x_i_rm.reshape(x_i.shape)

for i, x_i in enumerate(x_ts):
    x_i = x_i.squeeze().astype(np.uint8)
    _, output, stats, _ = cv2.connectedComponentsWithStats(x_i, connectivity=8)

    min_component_size = 50

    x_i_rm = np.zeros(output.shape)
    for j, size in enumerate(stats[1:, -1]):  # Ignore the background
        if size >= min_component_size:
            x_i_rm[output == j + 1] = 255
    x_ts[i] = x_i_rm.reshape(x_i.shape)

In [14]:
plt.figure()
plt.suptitle('Random samples before removing dots')
for i in range(num_random_samples[0]):
    for j in range(num_random_samples[1]):
        plt.subplot(num_random_samples[0], num_random_samples[1], i * num_random_samples[1] + j + 1)
        plt.imshow(x_tr_org[idx[i, j]].squeeze().squeeze(), cmap='gray')
plt.show()

plt.figure()
plt.suptitle('Random samples after removing dots')
for i in range(num_random_samples[0]):
    for j in range(num_random_samples[1]):
        plt.subplot(num_random_samples[0], num_random_samples[1], i * num_random_samples[1] + j + 1)
        plt.imshow(x_tr[idx[i, j]].squeeze().squeeze(), cmap='gray')
plt.show()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

In [15]:
del x_tr_org

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

Device: cpu


In [17]:
class CustomVGG(nn.Module):
    def __init__(self, features, avgpool_size, num_classes, init_weights=True):
        super(CustomVGG, self).__init__()  # Initialize
        self.features = features
        self.avgpool = nn.AdaptiveAvgPool2d(avgpool_size)
        self.classifier = nn.Sequential(
            nn.Linear(128 * avgpool_size[0] * avgpool_size[1], 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 512),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, num_classes)
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view((x.size(0), -1))
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

In [18]:
def make_layers(cfg, in_channels=1, batch_norm=False):
    layers = []
    for l in cfg:
        if l == 'M':
            layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
        else:
            layers.append(nn.Conv2d(in_channels, l, kernel_size=3, padding=1))
            if batch_norm:
                layers.append(nn.BatchNorm2d(l))
            layers.append(nn.ReLU(inplace=True))
            in_channels = l
    return nn.Sequential(*layers)

In [19]:
def get_resnet(model_name='resnet152'):
    resnet = getattr(models, model_name)(pretrained=True)  # 1) Load pre-trained model
    for param in resnet.parameters():
        param.requires_grad = False  # 2) Freeze the layers
    in_features = resnet.fc.in_features
    resnet.fc = nn.Linear(in_features, num_classes)  # 3) Change the last layer of classifier
    resnet = resnet.double().to(device=device)
    return resnet, resnet.fc.parameters()

In [20]:
def get_alexnet():
    alexnet = models.alexnet(pretrained=True)  # 1) Load pre-trained model
    for param in alexnet.parameters():
        param.requires_grad = False  # 2) Freeze the layers
    in_features = alexnet.classifier[6].in_features
    alexnet.classifier[6] = nn.Linear(in_features, num_classes)  # 3) Change the last layer of classifier
    alexnet = alexnet.double().to(device=device)
    return alexnet, alexnet.classifier[6].parameters()

In [21]:
def get_vgg(model_name='vgg19_bn'):
    vgg = getattr(models, model_name)(pretrained=True)  # 1) Load pre-trained model
    for param in vgg.parameters():
        param.requires_grad = False  # 2) Freeze the layers
    in_features = vgg.classifier[6].in_features
    vgg.classifier[6] = nn.Linear(in_features, num_classes)  # 3) Change the last layer of classifier
    vgg = vgg.double().to(device=device)
    return vgg, vgg.classifier[6].parameters()

In [22]:
def get_densenet(model_name='densenet201'):
    densenet = getattr(models, model_name)(pretrained=True)  # 1) Load pre-trained model
    for param in densenet.parameters():
        param.requires_grad = False  # 2) Freeze the layers
    in_features = densenet.classifier.in_features
    densenet.classifier = nn.Linear(in_features, num_classes)  # 3) Change the last layer of classifier
    densenet = densenet.double().to(device=device)
    return densenet, densenet.classifier.parameters()

In [23]:
def get_squeezenet(model_name='squeezenet1_1'):
    squeezenet = getattr(models, model_name)(pretrained=True)  # 1) Load pre-trained model
    for param in squeezenet.parameters():
        param.requires_grad = False  # 2) Freeze the layers
    in_channels = squeezenet.classifier[1].in_channels
    squeezenet.classifier[1] = nn.Conv2d(in_channels,
                                         num_classes,
                                         kernel_size=(1,1),
                                         stride=(1,1))  # 3) Change the last layer of classifier
    squeezenet = squeezenet.double().to(device=device)
    return squeezenet, squeezenet.classifier[1].parameters()

In [24]:
vgg_cfg = [32, 32, 'M', 64, 64, 'M', 128, 128, 128, 'M'] #, 512, 512, 512, 'M', 512, 512, 512, 'M']
vgg_avgpool_size = (4, 4)

def get_custom_vgg():
    features = make_layers(vgg_cfg)
    custom_vgg = CustomVGG(features=features, num_classes=num_classes, avgpool_size=vgg_avgpool_size)
    custom_vgg = custom_vgg.float().to(device=device)
    return custom_vgg, custom_vgg.parameters()

In [25]:
def pre_trained_pad(x, lpad, rpad, tpad, bpad):
    x_pad = np.zeros((x.shape[0], x.shape[1], lpad + x.shape[2] + rpad, tpad + x.shape[3] + bpad))
    x_pad[:, :, lpad:-rpad, tpad:-bpad] = x
    return x_pad

In [26]:
def pre_trained_channel(x, num_channels):
    x_channel = np.concatenate([x] * num_channels, axis=1)
    return x_channel

In [27]:
num_tests = 1
test_size = 0.2

epochs = 200
mini_batch_size = 64
mini_batch_size_cv = 64
mini_batch_size_ts = 50

checkpoint_epochs = 5
cross_validation_epochs = 5

checkpoint_path = './checkpoints/{}_e{}.pt'
result_path = './results/{}.csv'

In [28]:
def cross_validate(x_tr, y_tr, clf, params, clf_name):
    tr_loss_hist, cv_loss_hist = [], []
    tr_acc_hist, cv_acc_hist = [], []

    tr_idx, cv_idx = next(StratifiedShuffleSplit(n_splits=1, test_size=test_size).split(x_tr, y_tr))

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(params)

    for epoch in range(epochs):
        clf.train()

        running_loss, running_corrects = 0, 0
        num_mini_batches = tr_idx.shape[0] / mini_batch_size
        for mini_batch, mini_batch_idx in enumerate(np.split(tr_idx, num_mini_batches)):
            x_mini_batch = torch.from_numpy(x_tr[mini_batch_idx]).to(device)
            y_mini_batch = torch.from_numpy(y_tr[mini_batch_idx]).long().to(device)

            y_pred = clf(x_mini_batch).to(device=device)

            loss = criterion(y_pred, y_mini_batch)
            running_loss += loss.data.item()

            _, y_pred = torch.max(y_pred, 1)
            y_pred = y_pred.to(device=device)
            corrects = (y_pred == y_mini_batch).float().to(device).sum()
            running_corrects += corrects

            optimizer.zero_grad()  # Reset gradients so it wouldn't be accumulated
            loss.backward()  # Calculate gradients
            optimizer.step()  # Update the weights

        accuracy = running_corrects / tr_idx.shape[0]
        logger.info(f'Epoch: {epoch + 1}/{epochs}, Accuracy: {accuracy}, Loss: {running_loss}')
        tr_loss_hist.append(running_loss)
        tr_acc_hist.append(accuracy)

        if (epoch + 1) % checkpoint_epochs == 0:
            torch.save(clf, checkpoint_path.format(clf_name, epoch + 1))

        if (epoch + 1) % cross_validation_epochs == 0:
            clf.eval()

            running_loss, running_corrects = 0, 0
            num_mini_batches = cv_idx.shape[0] / mini_batch_size_cv
            for mini_batch, mini_batch_idx in enumerate(np.split(cv_idx, num_mini_batches)):
                x_mini_batch = torch.from_numpy(x_tr[mini_batch_idx]).to(device)
                y_mini_batch = torch.from_numpy(y_tr[mini_batch_idx]).long().to(device)

                y_pred = clf(x_mini_batch).to(device=device)

                loss = criterion(y_pred, y_mini_batch)
                running_loss += loss.data.item()

                _, y_pred = torch.max(y_pred, 1)
                y_pred = y_pred.to(device=device)
                corrects = (y_pred == y_mini_batch).float().to(device).sum()
                running_corrects += corrects

            accuracy = running_corrects / cv_idx.shape[0]
            logger.info(f'Epoch: {epoch + 1}/{epochs}, CV Accuracy: {accuracy}, CV Loss: {running_loss}')
            cv_loss_hist.append(running_loss)
            cv_acc_hist.append(accuracy)
    return tr_loss_hist, cv_loss_hist, tr_acc_hist, cv_acc_hist

In [29]:
# NOTE: Only run this for pretrained models
# x_tr = pre_trained_pad(x_tr, pad, pad, pad, pad)
# x_tr = pre_trained_channel(x_tr, imagenet_img_channels)

In [30]:
clf_name = 'custom_vgg'
for _ in range(num_tests):
    clf, params = get_custom_vgg()
    tr_loss_hist, cv_loss_hist, tr_acc_hist, cv_acc_hist = cross_validate(x_tr, y_tr, clf, params, clf_name)

Epoch: 1/20, Accuracy: 0.0, Loss: 2.4136993885040283
Epoch: 2/20, Accuracy: 9.375000081490725e-05, Loss: 1.8941378593444824
Epoch: 3/20, Accuracy: 6.25000029685907e-05, Loss: 2.020894765853882
Epoch: 4/20, Accuracy: 9.375000081490725e-05, Loss: 1.4529719352722168
Epoch: 5/20, Accuracy: 9.375000081490725e-05, Loss: 1.5077224969863892
Epoch: 5/20, CV Accuracy: 0.0, CV Loss: 3.3445332050323486
Epoch: 6/20, Accuracy: 9.375000081490725e-05, Loss: 1.425131916999817
Epoch: 7/20, Accuracy: 9.375000081490725e-05, Loss: 1.4405416250228882
Epoch: 8/20, Accuracy: 0.00015624999650754035, Loss: 0.92940354347229
Epoch: 9/20, Accuracy: 0.0001875000016298145, Loss: 0.7765125036239624
Epoch: 10/20, Accuracy: 0.00015624999650754035, Loss: 1.098382830619812
Epoch: 10/20, CV Accuracy: 0.0, CV Loss: 8.196086883544922
Epoch: 11/20, Accuracy: 0.00021875000675208867, Loss: 1.223633050918579
Epoch: 12/20, Accuracy: 0.0001875000016298145, Loss: 0.705641508102417
Epoch: 13/20, Accuracy: 0.0001875000016298145, Los

In [31]:
plt.figure()
plt.title('Training Metrics')

plt.subplot(2, 1, 1)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(list(range(1, epochs + 1)), tr_loss_hist)
plt.plot(list(range(cross_validation_epochs, epochs + 1, cross_validation_epochs)), cv_loss_hist)
plt.legend(['train', 'cross validation'], loc='upper left')

plt.subplot(2, 1, 2)
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.plot(list(range(1, epochs + 1)), tr_loss_hist)
plt.plot(list(range(cross_validation_epochs, epochs + 1, cross_validation_epochs)), cv_acc_hist)
plt.legend(['train', 'cross validation'], loc='upper left')
plt.show()

FigureCanvasNbAgg()

In [32]:
def test(x_ts, clf_name, epoch):
    clf = torch.load(checkpoint_path.format(clf_name, epoch), map_location=device)
    clf.eval()
    with open(result_path.format(clf_name), 'w') as f:
        f.write('Id,Category\n')

        num_mini_batches = x_ts.shape[0] / mini_batch_size_ts
        for mini_batch, x_mini_batch in enumerate(np.split(x_ts, num_mini_batches)):
            x_mini_batch = torch.from_numpy(x_mini_batch).to(device)

            y_pred_oh = clf(x_mini_batch).to(device=device)  # Calculate output of test in one-hot format
            _, y_pred = torch.max(y_pred_oh, 1)  # Change from one-hot to class
            y_pred = y_pred.to(device=device)

            for i, y_pred_i in enumerate(y_pred):
                f.write('{},{}\n'.format(mini_batch * mini_batch_size_ts + i, y_pred_i))